Using GPT-3.5-turbo model via OpenAI API to classify and generate explanations for NLI samples from SI-NLI dataset

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from time import sleep
np.random.seed(424)

In [ ]:
import pickle

In [ ]:
# load SI-NLI dataset
train = pd.read_csv('data/SI-NLI/train.tsv', sep="\t")
dev = pd.read_csv('data/SI-NLI/dev.tsv', sep="\t")
test = pd.read_csv('data/SI-NLI/test.tsv', sep="\t")

In [ ]:
# select 100 samples from train set used to test the model
t = train.sample(n=100)
t['label'].value_counts()

In [ ]:
# load english translations of SI-NLI dataset
train_eng = pd.read_csv('data/SI-NLI-en/train.tsv', sep='\t')
t_eng = train_eng[train_eng['pair_id'].isin(t['pair_id'])]

In [ ]:
# only keep needed columns
t = t[['label', 'premise', 'hypothesis']]
t_eng = t_eng[['label', 'premise', 'hypothesis']]

### Prompts

In [ ]:
def prompt1si(premise, hypothesis):
	return f'Glede na stavek "{premise}" ugotovite, ali je naslednja izjava posledica, kontradikcija ali nevtralna: "{hypothesis}"\nOdgovor (posledica ali kontradikcija ali nevtralna) je:'

In [ ]:
def prompt1en(premise, hypothesis):
	return f'Given the sentence "{premise}", determine if the following statement is entailed or contradicted or neutral: "{hypothesis}"\nThe answer (entailed or contradicted or neutral) is:'

In [ ]:
def prompt2en(premise, hypothesis):
	return f'''Instructions: You will be presented with a premise and a hypothesis about that premise in slovene. You need to decide whether the hypothesis is entailed by the premise by choosing one of the following answers: 'entailment': The hypothesis follows logically from the information contained in the premise. 'contradiction': The hypothesis is logically false from the information contained in the premise. 'neutral': It is not possible to determine whether the hypothesis is true or false without further information. Read the passage of information thoroughly and select the correct answer from the three answer labels. Read the premise thoroughly to ensure you know what the premise entails.

Premise: {premise}
Hypothesis: {hypothesis}

Answer (just one word, either entailment/neutral/contradiction):'''

In [ ]:
def prompt2si(premise, hypothesis):
    return f'''Navodila: Predstavljena vam bo premisa in hipoteza o tej premisi. Odločiti se morate, ali je hipoteza posledica premise, tako da izberete enega od naslednjih odgovorov: 'posledica': Hipoteza logično sledi iz informacij, ki jih vsebuje premisa. 'kontradikcija': Hipoteza je logično napačna glede na informacije, ki jih vsebuje premisa. 'nevtralno': Brez dodatnih informacij ni mogoče ugotoviti, ali je hipoteza resnična ali napačna. Natančno preberite odlomek informacij in izberite pravilen odgovor med tremi oznakami odgovorov. Temeljito preberite predpostavko, da boste vedeli, kaj sledi iz predpostavke.

Premisa: {premise}
Hipoteza: {hypothesis}

Odgovor (samo ena beseda, bodisi posledica/nevtralno/kontradikcija):'''

Load OpenAI API

In [ ]:
import os
import openai

with open("API_KEY_OPENAI", encoding='utf-8') as f:
  openai.api_key = f.read()

In [ ]:
def gpt_request(prompt):
	response = openai.ChatCompletion.create(
		request_timeout=15,
		model="gpt-3.5-turbo",
		messages=[{'role': 'user', 'content': prompt}],
		temperature=0,
		max_tokens=20,
		stop=["\n"]
	)

	return response.choices[0].message.content, response.usage.total_tokens

### Helper functions

In [ ]:
def gpt_multiple_requests(df, prompt):
	responses = []
	tokens = []
	for i, row in tqdm(df.iterrows()):
		response, token = gpt_request(prompt(row['premise'], row['hypothesis']))
		responses.append(response)
		tokens.append(token)
	return responses, sum(tokens)

In [ ]:
# extract classification from responses
def process_slo(answer):
	answer = answer.lower()
	if 'posledica' in answer:
		return 'entailment'
	elif 'kontradikcija' in answer:
		return 'contradiction'
	elif 'nevtral' in answer:
		return 'neutral'
	else:
		return answer

def batch_process_slo(answers):
	return [process_slo(answer) for answer in answers]

def process_eng(answer):
	answer = answer.lower()
	if 'entail' in answer:
		return 'entailment'
	elif 'contradict':
		return 'contradiction'
	elif 'neutral' in answer:
		return 'neutral'
	else:
		return answer
	
def batch_process_eng(answers):
	return [process_eng(answer) for answer in answers]

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

def confusion_matrix_for_column(df, col):
	cm = confusion_matrix(df['label'], df[col], labels=['entailment', 'neutral', 'contradiction'])
	sns.heatmap(cm, annot=True, xticklabels=['entailment', 'neutral', 'contradiction'], yticklabels=['entailment', 'neutral', 'contradiction'])
	plt.xlabel('Predicted')
	plt.ylabel('True')
	plt.title('Confusion matrix for ' + col)
	plt.show()

# Zero shot

In [ ]:
responses = []
tokens = 0

In [ ]:
prompt = prompt1si # one of the four prompt functions to use
df = t # dataset to use

for i, row in tqdm(df.iterrows()):
	response, token = gpt_request(prompt(row['premise'], row['hypothesis']))
	responses.append(response)
	tokens += token

In [ ]:
responses_processed = batch_process_slo(responses) # extract classification from responses
t['prompt1si'] = responses_processed # add to dataframe

In [ ]:
# count how many correct
print(sum(t['label'] == t['prompt1si']))
print(t['prompt1si'].value_counts())

In [ ]:
tokens

In [ ]:
confusion_matrix_for_column(t, 'prompt1si')

# Few Shot

### Helper functions

In [ ]:
# selects k samples from each label randomly
def get_k_each(df, k, random_state=42):
    samples_e = df[df['label'] == 'entailment'].sample(k, random_state=random_state)
    samples_n = df[df['label'] == 'neutral'].sample(k, random_state=random_state)
    samples_c = df[df['label'] == 'contradiction'].sample(k, random_state=random_state)
    return pd.concat([samples_e, samples_n, samples_c]).sample(frac=1, random_state=random_state).reset_index(drop=True)

In [ ]:
map_label_to_eng = {'entailment': 'entailed', 'neutral': 'neutral', 'contradiction': 'contradicted'}

# generates prompt for few shot prompting from examples and premise, hypothesis
def few_shot_prompt(samples, premise, hypothesis):
	prompt = ''
	for	i, row in samples.iterrows():
		prompt += prompt1en(row['premise'], row['hypothesis'])
		prompt += map_label_to_eng[row['label']] + '\n\n'
	prompt += prompt1en(premise, hypothesis)
	return prompt

In [ ]:
# 3 selections for 3 shot prompting (1 example per class)
samples3_1 = get_k_each(train, 1, 42)
samples3_2 = get_k_each(train, 1, 52)
samples3_3 = get_k_each(train, 1, 62)

### Generating predictions

In [ ]:
responses = []
tokens = 0

In [ ]:
samples = samples3_1 # samples to use for few-shot learning
df = t # dataframe to use for testing
for i in tqdm(range(len(responses), len(df))):
	row = df.iloc[i]
	response, token = gpt_request(few_shot_prompt(samples, row['premise'], row['hypothesis']))
	responses.append(response)
	tokens += token
	sleep(0.5)

In [ ]:
responses_processed = batch_process_eng(responses)
t['few3_1'] = responses_processed

# count how many correct
print(sum(t['label'] == t['few3_1']))
print(t['few3_1'].value_counts())

In [ ]:
tokens

In [ ]:
confusion_matrix_for_column(t, 'few3_1')

# Explanations

### Helper functions

In [ ]:
def prompt_expl(premise, hypothesis):
	return f'''Given the sentence "{premise}", determine if the following statement is entailed or contradicted or neutral: "{hypothesis}". First give a short, one sentence reasoning or explanation for the decision in slovene, and then the final answer (one english word - "entailed" or "contradicted" or "neutral") in a new line after that.

Razlaga v slovenščini:'''

In [ ]:
def gpt_request_expl(prompt):
	response = openai.ChatCompletion.create(
		request_timeout=15,
		model="gpt-3.5-turbo",
		messages=[{'role': 'user', 'content': prompt}],
		temperature=0,
		max_tokens=512,
	)

	return response.choices[0].message.content, response.usage.total_tokens

In [ ]:
# extracts the classification from the response of the model
def process_eng_expl(response):
	answer = response.strip().split('\n')[-1]
	answer = answer.lower()
	if 'entail' in answer or 'potrjen' in answer:
		return 'entailment'
	elif 'contradict' in answer or 'kontra' in answer:
		return 'contradiction'
	elif 'neutral' in answer or 'nevt' in answer:
		return 'neutral'
	else:
		return answer
	

# extracts the explanation from the response of the model
def extract_explanation(response):
	response = '\n'.join(response.strip().split('\n')[:-1]).strip()
	return response
	
def batch_process_eng_expl_preds(answers):
	return [process_eng_expl(answer) for answer in answers]

def batch_process_eng_expl(answers):
	return [extract_explanation(answer) for answer in answers]

### Using the model

In [ ]:
responses = []
tokens = 0

In [ ]:
with open("temp", "rb") as fp:
	b = pickle.load(fp)

In [ ]:
with open("temp", "wb") as fp:
	pickle.dump(responses, fp)

In [ ]:
df = t
for i in tqdm(range(len(responses), len(df))):
	row = df.iloc[i]
	response, token = gpt_request_expl(prompt_expl(row['premise'], row['hypothesis']))
	responses.append(response)
	tokens += token
	
	with open("temp", "wb") as fp:
		pickle.dump(responses, fp)
	sleep(0.01)

responses_expl = responses

In [ ]:
responses_expl_preds_processed = batch_process_eng_expl_preds(responses_expl)
responses_expl_processed = batch_process_eng_expl(responses_expl)
t['expl_preds'] = responses_expl_preds_processed
t['expl'] = responses_expl_processed

# count how many correct
print(sum(t['label'] == t['expl_preds']))
print(t['expl_preds'].value_counts())

In [ ]:
confusion_matrix_for_column(t, 'expl_preds')

Selects 50 samples from the test set and saves them to a file. To use for grading explanations.

In [ ]:
sample_e = t[t['label'] == 'entailment'].sample(18, random_state=442)
sample_c = t[t['label'] == 'contradiction'].sample(16, random_state=442)
sample_n = t[t['label'] == 'neutral'].sample(16, random_state=442)
sample = pd.concat([sample_e, sample_c, sample_n]).sample(frac=1, random_state=442)

In [ ]:
sample.to_csv('data/explanations_sample.tsv', sep='\t', index=False)

# English zero shot

Using english translations of the data

In [ ]:
responses = []
tokens = 0

In [ ]:
prompt = prompt1en
df = t
for i, row in tqdm(df.iterrows()):
	response, token = gpt_request(prompt(row['premise'], row['hypothesis']))
	responses.append(response)
	tokens += token

responses_1en = responses

In [ ]:
responses_1en_processed = batch_process_eng(responses_1en)

In [ ]:
t_eng['prompt1en_eng'] = responses_1en_processed

In [ ]:
# count how many correct
print(sum(t_eng['label'] == t_eng['prompt1en_eng']))
print(t_eng['prompt1en_eng'].value_counts())

In [ ]:
tokens

In [ ]:
confusion_matrix_for_column(t_eng, 'prompt1en_eng')